In [ ]:
# weather script
# aug 4th

# getting the code provided by the open meteo api

In [5]:
! pip install openmeteo-requests
! pip install requests-cache retry-requests numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.9/669.9 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.6/635.6 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 53.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry
from datetime import datetime


In [ ]:
# setup the api client with cache and retry on error

cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


In [8]:
# weather variables

url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 40.74,
	"longitude": -73.89,
	"daily": ["weather_code", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "uv_index_max", "precipitation_sum", "precipitation_hours", "precipitation_probability_max", "apparent_temperature_max", "apparent_temperature_min", "temperature_2m_max", "temperature_2m_min"],
	"hourly": ["temperature_2m", "apparent_temperature", "precipitation", "cloud_cover", "visibility", "soil_temperature_0cm", "is_day", "sunshine_duration"],
	"current": ["temperature_2m", "apparent_temperature", "is_day", "precipitation", "weather_code"],
}
responses = openmeteo.weather_api(url, params=params)


In [9]:
responses

In [13]:
# For one location

response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")


Coordinates: 40.742000579833984°N -73.87183380126953°E
Elevation: 26.0 m asl
Timezone difference to GMT+0: 0s


In [24]:
# process current data
# the order of variables needs to be the same as requested.

current = response.Current()
current_temperature_2m = current.Variables(0).Value()
current_apparent_temperature = current.Variables(1).Value()
current_is_day = current.Variables(2).Value()
current_precipitation = current.Variables(3).Value()
current_weather_code = current.Variables(4).Value()

timestamp = current.Time()
readable_time = datetime.utcfromtimestamp(timestamp)

print(f"Coordinates: {response.Latitude():.2f}°N {response.Longitude():.2f}°E")
print(f"\nCurrent time: {readable_time}")
print(f"Current temperature_2m: {current_temperature_2m:.2f}")
print(f"Current apparent_temperature: {current_apparent_temperature:.2f}")
print(f"Current is_day: {current_is_day}")
print(f"Current precipitation: {current_precipitation}")
print(f"Current weather_code: {current_weather_code}")


Coordinates: 40.74°N -73.87°E

Current time: 2025-08-04 19:15:00
Current temperature_2m: 31.72
Current apparent_temperature: 32.36
Current is_day: 1.0
Current precipitation: 0.0
Current weather_code: 0.0


/var/folders/dz/qbfft_n92gx2ky_gxnc3q9_h0000gn/T/ipykernel_52414/504825277.py:12: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  readable_time = datetime.utcfromtimestamp(timestamp)
